# Make RGB NIRCam stamps showing slit footprint
We will be using the `trilogy` package to make RGB stamps and the shutter footprint regions.
The information we need to show the footprint and the source (optional) is all in the "shutters" files.


In [ ]:
from trilogy import trilogy
from astropy.io import fits
from astropy.wcs import WCS
from astropy.table import Table
from astropy import table
from astropy.visualization import make_lupton_rgb

from astropy.coordinates import SkyCoord
import astropy.units as u
import glob

from collections import defaultdict
import matplotlib
from matplotlib import pyplot as plt
import PIL  # Python Image Library
from PIL import Image
import os, sys
import contextlib

from astropy.nddata import Cutout2D
import numpy as np
from scipy.ndimage import zoom
import pandas as pd
import scipy
import os
import glob
import json
import numpy as np


Function to produce the RGB-stamps

In [ ]:
def normalise_image(img, stretch='asinh', Q=10, alpha=1, weight=1.0):
    """
    Normalises the input image with optional stretching and channel weighting.

    Parameters:
    - img: 2D numpy array
    - stretch: Type of stretch ('asinh' or 'linear')
    - Q: Controls asinh stretch strength
    - alpha: Controls non-linearity for asinh
    - weight: Multiplier to boost/dampen this channel’s contribution

    Returns:
    - Normalised image scaled between 0 and 1
    """
    
    # Replace nans with 0.0
    img = np.nan_to_num(img, nan=0.0, posinf=0.0, neginf=0.0)
    
    # Clip possible negative fluxes
    img = np.clip(img, 0, None)
    
    # Apply scaling weight
    img *= weight
    
    # Determine which scaling to us
    if stretch == 'asinh':  # Lupton scaling
        img_scaled = np.arcsinh(alpha * Q * img) / Q
    elif stretch == 'log':
        img_scaled = np.log10(1 + alpha * img)
    elif stretch == 'linear':
        img_scaled = img
    else:
        raise ValueError("Unknown stretch")
    
    # After stretching the image is normalised to 1
    return img_scaled / np.nanmax(img_scaled) if np.nanmax(img_scaled) != 0 else img_scaled


def preprocess_fits_image(filename, ext=0, stretch='asinh', Q=10, alpha=1, weight=1):
    """
    Load, optionally resample, and normalise a FITS image.

    Parameters:
    - filename: FITS filename with optional extension (e.g., 'file.fits[1]')
    - stretch, Q, alpha, weight: Passed to normalize_image
    - upscale_size: Tuple of (new_y, new_x) size to resize image to

    Returns:
    - Processed 2D numpy image
    """
    
    with fits.open(filename) as hdul:
        img_data = hdul[ext].data
    return normalise_image(img_data, stretch=stretch, Q=Q, alpha=alpha, weight=weight)


def make_stamp(imagesRGB, Q_r, alpha_r, Q_g, alpha_g, Q_b, alpha_b, outfile='stamp.pdf'):
    """
    Make RGB stamp using trilogy
    """
    
    ##################
    # MAKE RGB IMAGE #
    ##################
    
    # Process images for scaling purposes
    processed_images = {}
    for colour in ['R', 'G', 'B']:
        processed_images[colour] = []
        for image_str in imagesRGB[colour]:
            # Obtain filename and extension that carries the data
            fname, ext = image_str.split('[')
            ext = int(ext.replace(']', ''))
            if colour == 'R':
                norm_img = preprocess_fits_image(fname, ext, Q=Q_r, alpha=alpha_r)
            elif colour == "G":
                norm_img = preprocess_fits_image(fname, ext, Q=Q_g, alpha=alpha_g)
            else:
                norm_img = preprocess_fits_image(fname, ext, Q=Q_b, alpha=alpha_b)
                
            processed_images[colour].append(norm_img)

    # Add scaled images to the dictionary
    temp_files = {}
    for colour in processed_images:
        fname = f'temp_{colour}.fits'
        fits.writeto(fname, processed_images[colour][0], overwrite=True)
        temp_files[colour] = fname
    
    # set luminosity of the noise in each channel
    noiselums = {'R': 0.2, 'G': 0.2, 'B': 0.2}
        
    with open(os.devnull, "w") as fnull, contextlib.redirect_stdout(fnull): # avoid printing out stuff
        trilogy.Trilogy(
            infile=None, samplesize=20000, stampsize=20000, maxstampsize=20000,
            deletetests=1, deletefilters=1, testfirst=0, showwith="PIL",
            mode='RGB', imagesorder='RGB',
            imagesRGB={'R': [temp_files['R']], 'G': [temp_files['G']], 'B': [temp_files['B']]},
            noiselums=noiselums, images=None, outname='color_image_temp', satpercent=0.000, 
            noiselum=0.5, noisesig0=10, correctbias=0, colorsatfac=1, combine='sum', show=False
            ).run()
        
    # read in RGB image and delete png file
    im = Image.open('color_image_temp.png')
    im = im.transpose(method=Image.FLIP_TOP_BOTTOM)
    NIRCam_image = np.asarray(im)
    os.remove('color_image_temp.png')
    
    # read WCS from a single FITS file
    single_filename = imagesRGB['R'][0].split('[')[0] # remove '[1]' if present
    image_hdulist = fits.open(single_filename)
    image_wcs = WCS(image_hdulist[1].header, image_hdulist)
    
    # --- Plot using matplotlib and save ---
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.imshow(NIRCam_image, origin='lower')
    ax.axis('off')  # Hide axes
    
    # Save the RGB image as PDF
    fig.savefig(outfile, bbox_inches='tight', pad_inches=0.0)
    plt.close(fig)  # Clean up the figure




## Start making the stamps

In [ ]:

# ---------- SETUP ----------
input_dir = "/home/bpc/University/master/Red_Cardinal/cutouts_3x3/"
output_dir = "/home/bpc/University/master/Red_Cardinal/stamps/"
os.makedirs(output_dir, exist_ok=True)

# ---------- STEP 1: Get IDs from NIRCam, PRIMER and COSMOS-Web ----------
f444w_files = glob.glob(os.path.join(input_dir, "*F444W*.fits"))
f444w_ids = [os.path.basename(f).split("_F444W")[0] for f in f444w_files]

f770w_primer_files = glob.glob(os.path.join(input_dir, "*F770W_cutout_primer.fits"))
f770w_primer_ids = [os.path.basename(f).split("_F770W")[0] for f in f770w_primer_files]

f770w_cweb_files = glob.glob(os.path.join(input_dir, "*F770W_cutout_cweb.fits"))
f770w_cweb_ids = [os.path.basename(f).split("_F770W")[0] for f in f770w_cweb_files]

f1800w_files = glob.glob(os.path.join(input_dir, "*F1800W*.fits"))
f1800w_ids = [os.path.basename(f).split("_F1800W")[0] for f in f1800w_files]

# Remove all galaxies in f770w_primer_ids from f770w_cweb_ids
f770w_cweb_ids = np.setdiff1d(f770w_cweb_ids, f770w_primer_ids)


# ---------- STEP 2: Collect MIRI and NIRCam Files ----------
fits_files = glob.glob(os.path.join(input_dir, "*.fits"))
filters = ["F444W", "F770W", "F1800W"]


Function which calls the make_stamps function iteratively

In [ ]:

gal_id = "18769"
try:
    print(f"Creating image for {gal_id}...")
    
    # Construct base filenames (without [0])
    r_file_base = os.path.join(input_dir, f"{gal_id}_F1800W_cutout_primer.fits")
    g_file_primer_base = os.path.join(input_dir, f"{gal_id}_F770W_cutout_primer.fits")
    g_file_cweb_base = os.path.join(input_dir, f"{gal_id}_F770W_cutout_cweb.fits")
    b_file_base = os.path.join(input_dir, f"{gal_id}_F444W_cutout_nircam.fits")

    # Decide which F770W file to use
    if gal_id in f770w_primer_ids and os.path.exists(g_file_primer_base):
        g_file = g_file_primer_base + "[1]"
    elif gal_id in f770w_cweb_ids and os.path.exists(g_file_cweb_base):
        g_file = g_file_cweb_base + "[1]"
    else:
        print(f"[Skipping {gal_id}] No valid F770W file found.")
        #continue

    # Now safely add [0] to files
    r_file = r_file_base + "[1]"
    b_file = b_file_base + "[0]"
    
    # Stack into imagesRGB dictionary
    imagesRGB = {'R': [r_file], 
                 'G': [g_file], 
                 'B': [b_file]}
    
    # Call your image function (without shutters)
    # Q determines the strength of the non-linearity for the stretch.
    # alpha adjusts the sensitivity of the stretch for the given filter.

    outfile = os.path.join(output_dir, f"{gal_id}_stamp.pdf")
    make_stamp(imagesRGB, outfile=outfile, 
               Q_r=10, alpha_r=1.0,
               Q_g=10, alpha_g=1.0,
               Q_b=100, alpha_b=1.0)

except Exception as e:
    print(f"Error processing {gal_id}: {e}")


In [ ]:
nircam_fits = "/home/bpc/University/master/Red_Cardinal/cutouts_3x3/8338_F444W_cutout_3x3.fits"
miri_770_fits = "/home/bpc/University/master/Red_Cardinal/cutouts_3x3/8338_F770W_cutout_primer004.fits"
miri_1800_fits = "/home/bpc/University/master/Red_Cardinal/cutouts_3x3/8338_F1800W_cutout_primer004.fits"

with fits.open(nircam_fits) as nircam_hdul:
    nircam_data = nircam_hdul[0].data
    print(nircam_data)

with fits.open(miri_770_fits) as miri_770_hdul:
    miri_770_data = miri_770_hdul[1].data
    print(miri_770_data)

with fits.open(miri_1800_fits) as miri_1800_hdul:
    miri_1800_data = miri_1800_hdul[1].data
    print(miri_1800_data)

Check extensions on cutout FITS files

In [ ]:
r_file = "/home/bpc/University/master/Red_Cardinal/cutouts_3x3/18769_F1800W_cutout_primer.fits"
with fits.open(r_file) as hdul:
    for i, hdu in enumerate(hdul):
        print(f"HDU {i}: {hdu.name}, shape = {getattr(hdu.data, 'shape', None)}")

g_file = "/home/bpc/University/master/Red_Cardinal/cutouts_3x3/18769_F770W_cutout_primer.fits"
with fits.open(g_file) as hdul:
    for i, hdu in enumerate(hdul):
        print(f"HDU {i}: {hdu.name}, shape = {getattr(hdu.data, 'shape', None)}")
        
b_file = "/home/bpc/University/master/Red_Cardinal/cutouts_3x3/18769_F444W_cutout_nircam.fits"
with fits.open(b_file) as hdul:
    for i, hdu in enumerate(hdul):
        print(f"HDU {i}: {hdu.name}, shape = {getattr(hdu.data, 'shape', None)}")

In [ ]:
# read in shutter footprints
import regions, pickle
with open('shutters_bluejay/shutters_regions.pkl', 'rb') as f:
    shutters = pickle.load(f)
print(len(shutters))


In [ ]:
# loop through the list of shutters
for s in shutters:

    # read shutters into regions objects
    reg_source = regions.Regions.parse(s['source'], format='ds9')
    reg_source = reg_source[0] # list of regions with only one entry
    reg_aperture = regions.Regions.parse(s['AB_aperture'], format='ds9')
    
    # read FITS files with NIRCam cutouts
    imagesRGB = {"R": [f"v1.0_JWST_cutouts/{s['ID']}_F444W_cutout.fits[1]"], \
                 "G": [f"v1.0_JWST_cutouts/{s['ID']}_F200W_cutout.fits[1]"], \
                 "B": [f"v1.0_JWST_cutouts/{s['ID']}_F115W_cutout.fits[1]"]}
    
    # make footprint
    make_stamp_with_shutters(imagesRGB, reg_source, reg_aperture,
                             outfile=f"stamps/{s['ID']}.pdf", sourcemark=False, size_arcsec=3.0)

